# LangChain 核心模块 Agent - OpenAI Function


In [1]:
! pip install -U langchain

In [2]:
from langchain_openai import ChatOpenAI

# 使用 GPT-3.5-turbo
llm = ChatOpenAI(temperature=0)

In [3]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [4]:
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent

system_message = SystemMessage(content="你是非常强大的AI助手，但在计算单词长度方面不擅长。")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

In [5]:
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

C:\Users\lenovo\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIFunctionsAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use create_openai_functions_agent instead.
  warn_deprecated(


In [6]:
from langchain.agents import AgentExecutor

# 实例化 OpenAIFunctionsAgent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke("单词“educa”中有多少个字母?")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5单词“educa”中有5个字母。

> Finished chain.


{'input': '单词“educa”中有多少个字母?', 'output': '单词“educa”中有5个字母。'}

In [8]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=MEMORY_KEY)]
)

In [9]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

In [10]:
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

In [11]:
agent_executor.invoke("单词“educa”中有多少个字母?")



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5单词“educa”中有5个字母。

> Finished chain.


{'input': '单词“educa”中有多少个字母?',
 'chat_history': [HumanMessage(content='单词“educa”中有多少个字母?'),
  AIMessage(content='单词“educa”中有5个字母。')],
 'output': '单词“educa”中有5个字母。'}

In [12]:
agent_executor.invoke("那是一个真实的单词吗？")



> Entering new AgentExecutor chain...
抱歉，我无法确定“educa”是否是一个真实的单词。您可以查阅词典或在线搜索以确认该单词的存在。

> Finished chain.


{'input': '那是一个真实的单词吗？',
 'chat_history': [HumanMessage(content='单词“educa”中有多少个字母?'),
  AIMessage(content='单词“educa”中有5个字母。'),
  HumanMessage(content='那是一个真实的单词吗？'),
  AIMessage(content='抱歉，我无法确定“educa”是否是一个真实的单词。您可以查阅词典或在线搜索以确认该单词的存在。')],
 'output': '抱歉，我无法确定“educa”是否是一个真实的单词。您可以查阅词典或在线搜索以确认该单词的存在。'}